<a href="https://colab.research.google.com/github/msalmanz93/Zana-Coding-Challenge/blob/master/Task2_Zana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Install tflearn
!pip install tflearn

     |████████████████████████████████| 102kB 2.4MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=8ed85d00b81061d897dfb2341f5d37837760e08829579057fdf0f0ec470d093c
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [0]:
#Older version of tensorflow is required since some modules of tflearn isn't supported by  
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [0]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 52kB/s 
     |████████████████████████████████| 491kB 44.8MB/s 
     |████████████████████████████████| 3.2MB 48.6MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
from google.colab import files
uploaded = files.upload()

Saving intent.json to intent.json


In [0]:
# import the json intent file
import json
with open('intent.json') as json_data:
    intents = json.load(json_data)

Preparing Data
(1) Tokenise patterns into array of words
(2) Lower case and stem all words. Example: Pharmacy = pharm. Attempt to represent related words
(3) Create list of classes — intents
(4) Create list of documents — combination between list of patterns and list of intents

In [0]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


Training Neural Network 
>> (1) Define X input shape = equal to word vocabulary size (2) Define two layers with 8 hidden neurones — optimal for text classification task (based on experiments) (3) Define Y input shape — equal to number of intents

In [0]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

(4) Apply regression to find the best equation parameters
(5) Run model.fit to construct classification model. Provide X/Y inputs, number of epochs and batch size
(6) Per each epoch, multiple operations are executed to find optimal model parameters to classify future input converted to array of 0/1

In [0]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network - input data shape, number of words in vocabulary (size of first array element). 
net = tflearn.input_data(shape=[None, len(train_x[0])])
# Two fully connected layers with 8 hidden units/neurons - optimal for this task
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
# number of intents, columns in the matrix train_y
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
# regression to find best parameters, during training
net = tflearn.regression(net)

# Define Deep Neural Network model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_redsamurai_medical_logs')
# Start training (apply gradient descent algorithm)
# n_epoch - number of epoch to run
# Batch size defines number of samples that going to be propagated through the network.
model.fit(train_x, train_y, n_epoch=1000, batch_size=5, show_metric=True)
model.save('chatbot_redsamurai_medical_model.tflearn')

Training Step: 8999  | total loss: 0.21150 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.21150 - acc: 0.9882 -- iter: 40/45
Training Step: 9000  | total loss: 0.19273 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.19273 - acc: 0.9882 -- iter: 45/45
--
INFO:tensorflow:/content/chatbot_redsamurai_medical_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
p = bow("Load blood pessure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: blood
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [0]:
print(model.predict([p]))

[[5.6332779e-11 5.1041122e-04 7.9476154e-01 3.2390458e-06 1.6157302e-06
  2.0472319e-01 1.8833681e-13 7.8192126e-11 2.9850671e-08]]


In [0]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot_redsamurai_medical_training_data", "wb" ) )